In [2]:
using Gridap
using DelimitedFiles
using KrylovKit
using LinearAlgebra
using GLMakie
using Interpolations

In [3]:
EigMode = open(readdlm,"Eigenmode.txt")

A_x = EigMode[:,1]
knots = (A_x,)
Hzi = EigMode[:,2]
Eyi = EigMode[:,3]
Hz_eigy = Interpolations.interpolate(knots, Hzi, Gridded(Linear()))
Ey_eigy = Interpolations.interpolate(knots, Eyi, Gridded(Linear()))
Hz_eig(x) = Hz_eigy(x[2])*cis(-1.654*k*(x[1]))
Ey_eig(x) = Ey_eigy(x[2])*cis(-1.654*k*(x[1]))
#Hz_eig(x) = Hz_eigy(x[2])*cis(-2.854*k*(x[1]))
#Ey_eig(x) = Ey_eigy(x[2])*cis(-2.854*k*(x[1]))
Mode_norm = Hzi'*Eyi*(A_x[2]-A_x[1])
q_vec=VectorQ(Ey_eig,Mode_norm,gridap)
Mode_norm
#N = 201
#Larray = range(-0.5,0.5;length=N)
#H_itpr = zeros(N)
#for i=1:N
#    H_itpr[i] = itpr[Larray[i]]
#end

#lines(Larray,H_itpr)

LoadError: UndefVarError: VectorQ not defined

In [ ]:
#1:b,2:-a,3:a,s:-b
normal_vec = VectorValue(1,0)
Q_mat = assemble_matrix(gridap.FE_U,gridap.FE_V) do u,v
    ∫( -1im/4/k*((x->ξ0(x;phys))*((u*∇(v)-v*∇(u))⋅normal_vec)) )gridap.dΓ_t[1]
end
G_ii,V_raw,info = eigsolve(x->MatrixG(x;A_mat,D_mat,Q_mat),rand(ComplexF64,N),5,:LM;ishermitian=true)
sum((G_ii))
#u_vec'*Q_mat*u_vec